# Anomaly Detection in Public Company Financial Statements Using Density-Based Clustering

## Abstract

The Securities and Exchange Commission (SEC) requires all publicly held companies to submit financial statements on a quarterly and annual basis. The ability to identify anomalous submissions can help investors manage risk and the SEC uncover suspicious companies. However, manually retrieving and reviewing every public company’s financial statements in search of anomalies is an expensive process. To solve that problem and fill a methodological gap in the literature, this paper uses the density-based OPTICS algorithm to cluster and detect anomalous U.S. public company financial statements. The raw data were retrieved from the SEC’s APIs and consist of the most commonly disclosed financial metrics of 2020. The data were normalized, imputed, and decomposed into their principal components prior to modeling. We used OPTICS’ DBSCAN-like cluster extraction method, and 5-fold cross-validation to find the hyperparameter values yielding the highest Silhouette coefficient of 0.77. The final model classified 99.0% of companies as normal and 1.0% as anomalous, which we consider to be a reasonable distribution. Finally, we deployed this model as an interactive web application that investors and auditors can use to quickly discover anomalous companies.

## Table of Contents

1. [Introduction](#1.-Introduction)  
    * [1.1 Background](#1.1-Background)  
    * [1.2 Problem Statement](#1.2-Problem-Statement)     
    * [1.3 Objectives](#1.3-Objectives)    
2. [Literature Review](#2.-Literature-Review)  
3. [Methodology](#3.-Methodology)  
    * [3.1 Data Acquisition and Aggregation](#3.1-Data-Acquisition-and-Aggregation)  
    * [3.2 Data Quality](#3.2-Data-Quality)  
    * [3.3 Exploratory Data Analysis](#3.3-Exploratory-Data-Analysis)  
    * [3.4 Feature Engineering](#3.4-Feature-Engineering)   
        * [3.4.1 Normalization](#3.4.1-Normalization)  
        * [3.4.2 Imputation](#3.4.2-Imputation)  
        * [3.4.3 Principal Component Analysis](#3.4.3-Principal-Component-Analysis)   
    * [3.5 Modeling](#3.5-Modeling)  
        * [3.5.1 Selection of Modeling Techniques](#3.5.1-Selection-of-Modeling-Techniques)  
        * [3.5.2 Test Design](#3.5.2-Test-Design)  
4. [Results and Findings](#4.-Results-and-Findings)  
    * [4.1 Evaluation of Results](#4.1-Evaluation-of-Results)  
5. [Discussion](#5.-Discussion)  
    * [5.1 Conclusion](#5.1-Conclusion)  
    * [5.2 Future Research](#5.2-Future-Research)  
6. [References](#6.-References)  

## 1. Introduction

### 1.1 Background

The SEC is an independent U.S. government agency in charge of oversight of the securities markets. Its primary goal is to protect investors, prevent fraud, and assure fair stock market trades (SEC, n.d.). All publicly traded companies are required by federal law to file quarterly (10Q) and annual (10K) reports with the SEC (Form and Content of Financial Statements, 2022). 

The ability to identify anomalies can help individual investors avoid investing in companies with higher financial risk. It can help broker-dealers invest client assets in better suited securities that carry less risk. Lastly, anomaly detection can help the SEC identify companies that might require an additional audit of their financial statements.

### 1.2 Problem Statement

Manually retrieving and reviewing every public company's financial statements to find anomalies is a very laborious process that also requires expertise in finance and accounting. 

**Individual Investors**  
For individual investors without any financial acumen or data-wrangling skills, reviewing SEC-filed financial statements for anomalous companies is a very challenging task. The ability to easily identify anomalous companies would save them time and reduce their need for financial expertise.

**Coporate Investors**  
Prospective customers expect their broker or financial advisor to instantaneously identify anomalous companies and answer questions about them. Having a tool at their disposal that clusters anomalous companies would provide value to both the advisor and consumer.

**The SEC**  
A tool for clustering companies on the basis of their submitted financials could allow agency analysts to quickly identify suspicious companies that may require closer review.

### 1.3 Objectives

1. Cluster public companies based on commonly disclosed financial metrics. 
2. Detect anomalous public companies given their commonly disclosed financial metrics. 

## 2. Literature Review

Existing research on clustering financial statements for auditing and investing purposes focuses on distance, partition, hierarchical, and neural-network-based methods. One of the main challenges of this research is balancing model flexibility and interpretability. Flexibility is required to accurately capture patterns in complex multidimensional data, and to lessen the burden of prior knowledge placed on the end user. Intepretability, on the other hand, is necessary for the end user, whether it be an auditor or prospective investor, to trust the results enough to actually use them. The present study attempts to address this challenge and fill a methodological gap in the existing literature by exploring density-based clustering methods, which have been shown to perform well on other financial anomaly detection tasks (Ahmed et al., 2016). Specifically, this work uses the Ordering Points to Identify the Clustering Structure (OPTICS) algorithm to cluster U.S. public company financial statements and detect anomalies. OPTICS is flexible in that it can fit clusters of arbitrary shape and represent clustering information for a wide range of hyperparameter values. Furthermore, its resulting reachability plot provides intuitive cluster interpretation, especially when accompanied by a corresponding attribute plot (Ankerst et al., 1999). 

## 3. Methodology

This project uses the density-based OPTICS algorithm to cluster and detect anomalous U.S. public company financial statements. The raw data were retrieved from the SEC's APIs and consist of the most commonly disclosed financial metrics of 2020. We explored the data for patterns and issues, which informed our preprocessing decisions. The data were normalized, imputed, and decomposed into their principal components prior to modeling. We used the following Python libraries to implement our methodology: requests, numpy, pandas, matplotlib, seaborn, pca, scikit-learn, and streamlit. 

### 3.1 Data Acquisition and Aggregation

Data were acquired via the SEC's Extensible Business Markup Language (XBML) APIs. These APIs contain data on thousands of distinct facts, but were were only interested in the most cmmonly reported financial ones. To identify the most prevalent metrics, we randomly sampled 100 companies from the Company Facts API (SEC, 2022a). The sample companies have collectively disclosed 5,305 unique facts, but we only retained the 46 facts reported by the majority of companies in the sample. Data on those metrics for the 2020 calendar year were then retrieved from the Frames API, though three of them had no available data (SEC, 2022b). This resulted in a dataset of 7,683 companies that disclosed at least one of the 43 facts for the 2020 calendar year. Figure 1 demonstrates the data acquisition process. Once acquired, the data were then randomly split into a training (80%) and test (20%) set. Although this work involves unsupervised learning of unlabeled data, the test set serves to ensure the methodology does not overfit to spurious clusters in the training set. 

In [1]:
from IPython.display import Image

print('Figure 1 \nData Acquisition Workflow')
Image(url='../figures/1-data-acquisition-workflow.png', width=750, height=750)

Figure 1 
Data Acquisition Workflow


### 3.2 Data Quality  

The primary data quality issue was the prevalance of missing values. We only retained variables missing fewer than 30% of their values, which included the following 11 variables: 
1. Current Assets 
2. Total Assets 
3. Cash and Cash Equivalents at Carrying Value 
4. Authorized Shares of Common Stock 
5. Issued Shares of Common Stock 
6. Common Stock Value
7. Liabilities and Stockholders Equity 
8. Current Liabilities 
9. Total Liabilities 
10. Retained Earnings (Accumulated Deficit)
11. Stockholders Equity 

### 3.3 Exploratory Data Analysis

We explored the univariate and bivariate distributions of the 11 variables and found them all to be extremely right skewed. Figure 2 shows the distribution of total assets. Futhermore, many of them were strongly positively correlated (See Figure 3). For example, Figure 4 illustrates the strong positive correlation between Retained Earnings (Accumulated Deficit) and Stockholders Equity. 

In [2]:
print('Figure 2 \nDistribution of Total Assets')
Image(url='../figures/2-distribution-of-total-assets.png', width=750, height=750)

Figure 2 
Distribution of Total Assets


In [3]:
print('Figure 3 \nCorrelation Matrix Heatmap')
Image(url='../figures/3-correlation-matrix-heatmap.png', width=750, height=750)

Figure 3 
Correlation Matrix Heatmap


In [4]:
print('Figure 4 \nScatterplot of Retained Earnings (Accumulated Deficit) and Stockholders Equity')
Image(url='../figures/4-scatterplot-of-retained-earnings-accumulated-deficit-and-stockholders-equity.png', width=750, height=750)

Figure 4 
Scatterplot of Retained Earnings (Accumulated Deficit) and Stockholders Equity


### 3.4 Feature Engineering

We applied preprocessing techniques to de-skew, center, and scale the features, impute missing values, and remove multicollinearity. These transformations were applied to the training and test datasets independently to avoid information leakage. 

#### 3.4.1 Normalization

We applied the Yeo-Johnson power transformation to each variable because it can handle negative values, unliked the Box-Cox transformation (Yeo & Johnson, 2000). This made features more normally distributed with a mean of 0 and a standard deviation of 1. Figure 5 shows the transformed distribution of total assets, which can be compared to its original distribution shown in Figure 2. 

In [5]:
print('Figure 5 \nCorrelation Matrix Heatmap')
Image(url='../figures/5-transformed-distribution-of-total-assets.png', width=750, height=750)

Figure 5 
Correlation Matrix Heatmap


#### 3.4.2 Imputation

We leveraged the strong mututal information among the features to impute their missing values. Specifically, each missing value was estimated from the Euclidean distance-weighted average of its five nearest neighbors, which has been shown to be a robust method (Troyanskaya et al., 2001). 

#### 3.4.3 Principal Component Analysis

We applied principal component analysis to remove multicollinearity among the features. The first six principal components explained 94.5% of the variance, and the remaining five were discarded. 

In [6]:
print('Figure 6 \nExplained Variance by Principal Component')
Image(url='../figures/6-explained-variance-by-principal-component.png', width=750, height=750)

Figure 6 
Explained Variance by Principal Component


Analysis of each component's loadings found them to be quite interpretable For example, PC1 was positively correlated with all of the features, and therefore appeared to represent overall company size. PC2 was most strongly correlated with stockholders' equity and retained earnings, so it could be understood as overall company value. Figure 7 shows a scatter plot of the first two principal components and their strongest loadings: total assets and stockholders' equity, respectively. We can see that most companies are densely distributed within this two-dimensional projection. 

In [7]:
print('Figure 7 \nScatterplot of First Two Principal Components')
Image(url='../figures/7-scatterplot-of-first-two-principal-components.png', width=750, height=750)

Figure 7 
Scatterplot of First Two Principal Components


### 3.5 Modeling

We used the density-based algorithm OPTICS to cluster the companies and detect anomalies. The training and test datasets were used for hyperparameter tuning and validation, respectively. The final model was retrained on the entire dataset, interpreted, and deployed to an interactive web application. 

#### 3.5.1 Selection of Modeling Techniques 
We decided to use the density-based OPTICS clustering algorithm for its flexibility and interpretability. It can identify clusters of arbitrary shape and has built-in anomaly detection. Furthermore, its reachability scores provide an intuitive representation of the clustering structure and anomaly strength (Ankerst et al., 1999). 

#### 3.5.2 Test Design
Because ground truth cluster and outlier labels were unavailable for this task, we relied on internal cluster validation methods. Specifically, we used the Silhoutte coefficient as our measure of cluster validity (Rousseeuw, 1987). We used 5-fold cross-validation on the training data to find the hyperparameter values that yielded the highest Silhouette coefficient. The hyperparameters we tuned were ɛ, which is the neighborhood radius in Euclidean distance, and *MinPts*, which is the number of neighbors required for a point to be considered "core" (Ankerst et al., 1999). 

Instead of using OPTICS' reachability slope criteria to extract clusters, we opted for its DBSCAN-like method because it allows the user to explicitly set and visualize the noise threshold on the y-axis of the reachability plot, which supports our goal of intuitive anomaly detection. 

## 4. Results and Findings

The hyperparameter values that yielded the highest 5-fold cross-validated Silhouette coefficient of 0.77 were ɛ = 3.5 and *MinPts* = 1% (See Figure 8). The model was then fit to the entire training dataset, placing 99.0% of the companies into a single normal cluster and classifying the remaining 1.0% as noise. The model was then fit to the test dataset, which yielded a Silhouette coefficient of 0.70 and labeled 99.2% and 0.8% of the companies as normal and noise, respectively. 

In [8]:
print('Figure 8 \nHyperparameter Tuning Results')
Image(url='../figures/8-hyperparameter-tuning-results.png', width=750, height=750)

Figure 8 
Hyperparameter Tuning Results


### 4.1 Evaluation of Results

The model's consistency across the training and test sets suggests it did not overfit, and its anomaly detection rate of roughly 1% seemed reasonable for our application. For those reasons we decided to retrain the model on the entire dataset and interpret the results. One of OPTICS' outputs is each point's reachability, which is its distance to its nearest neighbor (Ankerst et al., 1999). The reachability plot in Figure 9 shows how the vast majority of companies fell below ɛ and were fairly consistently reachable, while the small fraction that exceeded it were much more isolated. 

In [9]:
print('Figure 9 \nReachability Plot')
Image(url='../figures/9-reachability-plot.png', width=750, height=750)

Figure 9 
Reachability Plot


Figure 10 also provides some intuition behind the clustering results by visualizing the clustering results in the first two principal components. Within this two-dimensional space, we see the majority of the normal companies are densely clustered near the origin, with most of the anomalies further spread out. However, some anomalies lie close to the origin, which suggests they are outliers in one or more of the principal components not visualized here. 

In [10]:
print('Figure 10 \nScatterplot of Clustering Output in First Two Principal Components')
Image(url='../figures/10-scatterplot-of-clustering-output-in-first-two-principal-components.png', width=750, height=750)

Figure 10 
Scatterplot of Clustering Output in First Two Principal Components


After identifying the anomalies, we sorted them by their reachability scores to provide a prioritized list for manual investigation. The most anomalous company was the Federal National Mortgage Association, also known as Fannie Mae. Upon inspecting its features, we found that it was an extreme outlier in the fourth principal component, which was most strongly correlated with accumulated deficit (see Figure 11). According to our raw dataset, Fannie Mae reported an accumulated deficit of $108 billion dollars at the end of 2020, so it is unsuprising that it is anomalous within that dimension. Furthermore, the Federal Housing Finance Agency (FHFA) placed Fannie Mae into conservatorship in 2008 after concluding they would become insolvent (FHFA, 2022). This validates our previous interpretation of the fourth principal component as representing insolvency risk. 

In [11]:
print('Figure 11 \nFeatures of the Most Anomalous Company')
Image(url='../figures/11-features-of-the-most-anomalous-company.png', width=750, height=750)

Figure 11 
Features of the Most Anomalous Company


The model has been made publicly available as an interactive web application hosted at the following address: https://andrewabeles-anomalous-companies-app-hmfo7e.streamlit.app/. Users can choose which year's financial metrics they would like to analyze, and the application retrieves and processes the data following the methodology described in this paper. However, instead of using cross-validation to determine the optimal value of ɛ, the application lets users explore various thresholds to discover the clustering and anomaly detection results most useful to them. 

## 5. Discussion

As far as we know, this is the first study to use density-based clustering to detect anomalous U.S. company financial statements. Our simple interactive web application allows individual investors with limited financial knowledge to quickly discover anomalous companies. Additionally, they can set their own anomaly thresholds based on their unique investment goals. Similarly, financial advisors searching for their clients can use the tool and tailor it to each client's specific risk appetite. Lastly, the SEC can use the application to easily identify abnormal companies whose financials may require further review. 

### 5.1 Conclusion

We believe this study achieved its objectives to cluster U.S. public company financial statements and identify anomalies. The final model classified 99.0% of companies as normal and 1.0% as anomalous, which we consider to be a reasonable distribution. In addition, the companion web application empowers users to adjust this distribution to suit their needs. Most importantly, the companies classified as anomalous due appear to be unique, and in various ways. The anomalous companies vary substantially in metrics of size and financial performance, which shows the methodology is flexible enough to identify outliers across multiple dimensions. Furthermore, we found the six principal components that captured 94.5% of the variance to be quite interpretable higher-level representations of the 11 original features. The web application allows users to visualize where the anomalies lie within pairs of principal components, and to gain an intuitive understanding of what each component represents through a visualization of its loadings. Based on these findings, we believe our anomaly detection tool successfully balances flexiblity and interpretability. However, it is of course not without limitations. 

### 5.2 Future Research

Future studies could expand on our work by exploring other datasets, imputation methods, and models. The dataset used in this study consisted of the 11 most commonly disclosed financial metrics by U.S. public companies for the 2020 calendar year. A natural extension would be to include multiple years worth of data for each company, as it would be interesting to see how information on companies' changes in financial metrics over time impacts the clustering and anomaly detection results. In addition to financial metrics, U.S. public companies are also required to disclose information on executive compensation, transfers in ownership, and material changes. These additional data might reveal outliers in company behavior, not just performance. Regarding imputation, we estimated missing values by taking the Euclidean distance-weighted average of a company’s five nearest neighbors. This seemed to work well for our objectives, but additional work could be done to test other distance metrics, weighting strategies, and numbers of neighbors. Finally, future research should explore other density-based anomaly detection models, including OPTICS cluster extraction methods and hyperparameters not tested in this study.

## 6. References

Ahmed, M., Mahmood, A. N., & Islam, R. (2016). A survey of anomaly detection techniques in financial domain. *Future Generation Computer Systems, 55*(2016), 278–288. https://doi.org/10.1016/j.future.2015.01.001  

Alzamil, Z. S., Appelbaum, D., Glasgall, W., & Vasarhelyi, M. A. (2021). Applications of data analytics: Cluster analysis of not-for-profit data. *Journal of Information Systems, 35*(3), 199–221.  https://doi.org/10.2308/isys-2020-025  

Ankerst, M., Breunig, M. M., Kriegel, H., & Sander, J. (1999). OPTICS: Ordering points to identify the clustering structure. *ACM SIGMOD Record, 28*(2), 49–60. https://doi.org/10.1145/304181.304187  

Byrnes, P. E. (2019). Automated clustering for data analytics. *Journal of Emerging Technologies in Accounting, 16*(2), 43–58. https://doi.org/10.2308/jeta-52474  

Dameric, R. P., Garelli, R., & Resta, M. (2020). Neural networks in accounting: Clustering firm performance using financial reporting data. *Journal of Information Systems, 34*(2), 149–166.  https://doi.org/10.2308/isys-18-002 

Federal Housing Finance Agency. (2022, October 17). *History of Fannie Mae and Freddie Mac conservatorships*. https://www.fhfa.gov/Conservatorship/Pages/History-of-Fannie-Mae--Freddie-Conservatorships.aspx

Form and Content of Financial Statements, Securities Act of 1933, Securities Exchange Act of 1934, Investment Company Act of 1940, Investment Advisers Act of 1940, and Energy Policy and Conservation Act of 1975, 17 C.F.R. § 210 (2022). https://www.ecfr.gov/current/title-17/chapter-II/part-210 

Kaplan, A., & Bien, J. (2022). Interactive exploration of large dendrograms with prototypes. *American Statistician*. https://doi.org/10.1080/00031305.2022.2087734 

Lokanan, M., Tran, V., & Vuong, N. H. (2019). Detecting anomalies in financial statements using machine learning algorithm: The case of Vietnamese listed firms. *Asian Journal of Accounting Research, 4*(2), 181–201. https://doi.org/10.1108/AJAR-09-2018-0032 

Moldovan, D., & Moca, M. (2015). A clustering of listed companies considering corporate governance and financial variables. *International Conference Knowledge-Based Organization, 21*(2), 338–343. https://doi.org/10.1515/kbo-2015-0056 

Rousseeuw, P. J. (1987). Silhouettes: A graphical aid to the interpretation and validation of cluster analysis. *Journal of Computational and Applied Mathematics, 20*, 53–65. https://doi.org/10.1016/0377-0427(87)90125-7 

SEC. (n.d.). *About us*. https://www.sec.gov/about.shtml 

SEC. (2022a). *Company facts API* [Data set]. https://www.sec.gov/edgar/sec-api-documentation  

SEC. (2022b). *Frames API* [Data set]. https://www.sec.gov/edgar/sec-api-documentation 

Troyanskaya, O., Cantor, M., Sherlock, G., Brown, P., Hastie, T., Tibshirani, R., Botstein, D., & Altman, R. B. (2001). Missing value estimation methods for DNA microarrays. *Bioinformatics, 17*(6), 520–525. 

Yeo, I., & Johnson, R. A. (2000). A new family of power transformations to improve normality or symmetry. *Biometrika, 87*(4), 954–959. https://doi.org/10.1093/biomet/87.4.954